In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm

# Web-Scrapping

In [3]:
url = 'https://myanimelist.net/topanime.php'
limit = 50
params = [{'limit':limit*i} for i in range(0,20)]
soup = []

for i in range(0,20):
    page = requests.get(url, params=params[i])
        
    if page.status_code == 200:
        soup.append(BeautifulSoup(page.text, 'html'))
    else:
        print(f"Request failed")

Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed


In [3]:
## soup contains html of 21 pages of the Top Rated Anime Series of all Time (Top 1000 Anime)
len(soup)

20

### Target Table
###### < table border="0" cellpadding="0" cellspacing="0" width="100%" class="top-ranking-table" >

In [4]:
#### Fetching Table header

table = soup[0].find('table',class_='top-ranking-table')
header_element = table.find('tr', class_ = 'table-header')
header = header_element.find_all('td')

title = []
for head in header:
    title.append(head.text)
    
title = title[:3] 

In [5]:
title.extend(['popularity','Members','Type', 'Episodes', 'Genre', 'Demographic','Start Date', 'End Date',  'Image', 'Link'])
                             ####     ###     ####                                  ######      ######       ####   #####

### Creation of Data-Frame

In [6]:
df = pd.DataFrame(columns = title)
df

,Rank,Title,Score,popularity,Members,Type,Episodes,Genre,Demographic,Start Date,End Date,Image,Link


### Fetching Anime Data

In [157]:
Row_Master = []

In [159]:
for i in tqdm(range(0,1), desc=f"-------------Processing Page----------", unit="item"):
    
    table = soup[i].find('table',class_='top-ranking-table')
    row_data = soup[i].find_all('tr', class_='ranking-list')

    for i in tqdm(range(len(row_data)), desc="Processing Anime", unit="item"):
        ### Initializing default values in case Data is missing
        rank = 0
        title = ''
        score = 0
        popularity=0
        members=0
        anime_type=''
        episodes=0
        genre=''
        demographic=''
        start_date='' 
        end_date='' 
        img_value=''
        href_value=''


        rd = row_data[i].find_all('td')

        rank_tag = rd[0].find('span', class_ = 'lightLink')
        if rank_tag:
            rank =rank_tag.text 
            rank = int(rank)

        anchor_tag = rd[1].find('a')
        if anchor_tag:
            href_value = anchor_tag.get('href')



        img_tag = rd[1].find('img')
        if img_tag:
            img_value = img_tag.get('data-srcset').split(", ")
            img_value = [img[:-3] for img in img_value]
            img_value = img_value[1]

        name_div = rd[1].find('div', class_='di-ib')
        if name_div:
                title = name_div.find('a',class_='hoverinfo_trigger').text
                title = title.replace('"', '')
                

        detail_div = rd[1].find('div', class_='information')
        if detail_div:
                details = detail_div.text.split('\n')
                details = [det.strip() for det in details if det.strip()!='']



        score_div = rd[2].find('span')
        if score_div:
                score =score_div.text 
                score = float(score)

        res = requests.get(href_value)
        if res.status_code == 200:
            anime_soup = BeautifulSoup(res.text, 'html')
            table = anime_soup.find('table')
            table_row = table.find('tr')
            pop_div = table_row.find('div', class_='stats-block po-r clearfix')

            strong = pop_div.find('span', class_='numbers popularity')
            strong = strong.find_all('strong')
            popularity = strong[0].text[1:]
            popularity = int(popularity)
            
            left_div = anime_soup.find('div', class_='leftside')
            info_div = left_div.find_all('div', class_='spaceit_pad')

            for info in info_div:

                text = info.find('span').text
                if text == 'Demographic:':
                     demographic = info.find_all('span')[1].text
                elif text == 'Genres:' or text == 'Genre:':
                    genre = info.find_all('span')
                    genres = []
                    for gen in genre:
                        genres.append(gen.text)
                    genre = [gen.strip() for gen in genres]
                    genre=genre[1:]

        members = details[2]
        members = ''.join(filter(str.isdigit, members))
        members = int(members)


        anime_type_eps = details[0].split(' ')
        anime_type = anime_type_eps[0]
        episodes = anime_type_eps[1][1:]
        if episodes != '?':
            episodes = int(episodes)

        if "to" in details[1]:
            date_string = details[1]
            start_date, end_date = date_string.split(' to ')
        else:
            date_string = details[1]
            if len(date_string.split(' - ')) == 1:
                start_date,end_date = date_string.split(' -')
                start_date = datetime.strptime(start_date, '%b %Y').date()

            else:
                start_date, end_date = date_string.split(' - ')
                start_date = datetime.strptime(start_date, '%b %Y').date()
                end_date = datetime.strptime(end_date, '%b %Y').date()

        new_list = [rank,title,score,popularity, members,anime_type, episodes, genre, demographic,start_date, end_date, img_value, href_value]
        Row_Master.append(new_list)
        print(new_list)

Processing Anime:   4%|▍         | 2/50 [00:00<00:09,  4.84item/s]

[1, 'Fullmetal Alchemist: Brotherhood', 9.1, 0, 3246818, 'TV', 64, '', '', datetime.date(2009, 4, 1), datetime.date(2010, 7, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/1208/94745.jpg?s=5ec18639199f2c60b60009f34222228d', 'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood']
[2, 'Steins;Gate', 9.07, 0, 2494983, 'TV', 24, '', '', datetime.date(2011, 4, 1), datetime.date(2011, 9, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/1935/127974.jpg?s=a0c19821033ace3af10310a253a9e85b', 'https://myanimelist.net/anime/9253/Steins_Gate']



Processing Anime:   6%|▌         | 3/50 [00:00<00:09,  5.19item/s]


[3, 'Gintama°', 9.06, 0, 611603, 'TV', 51, '', '', datetime.date(2015, 4, 1), datetime.date(2016, 3, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/3/72078.jpg?s=bb26d45bb75e13601e1b8029f717c2b0', 'https://myanimelist.net/anime/28977/Gintama°']
[4, 'Gintama: The Final', 9.05, 0, 140720, 'Movie', 1, '', '', datetime.date(2021, 1, 1), datetime.date(2021, 1, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/1245/116760.jpg?s=8fc5f4b436ee45307a882948a5e768ac', 'https://myanimelist.net/anime/39486/Gintama__The_Final']


Processing Anime:  10%|█         | 5/50 [00:00<00:08,  5.43item/s]


...

[23, 'Owarimonogatari 2nd Season', 8.88, 0, 388396, 'TV', 7, '', '', datetime.date(2017, 8, 1), datetime.date(2017, 8, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/6/87322.jpg?s=5d0ef74fd5c7e44a1e695e85d0b86331', 'https://myanimelist.net/anime/35247/Owarimonogatari_2nd_Season']
[24, 'Gintama.: Shirogane no Tamashii-hen - Kouhan-sen', 8.88, 0, 191680, 'TV', 14, '', '', datetime.date(2018, 7, 1), datetime.date(2018, 10, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/1776/96566.jpg?s=a87ac154753a26bfed4dbd557a8544ba', 'https://myanimelist.net/anime/37491/Gintama__Shirogane_no_Tamashii-hen_-_Kouhan-sen']


Processing Anime:  52%|█████▏    | 26/50 [00:04<00:04,  5.63item/s]

[25, 'Monster', 8.87, 0, 1060198, 'TV', 74, '', '', datetime.date(2004, 4, 1), datetime.date(2005, 9, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/10/18793.jpg?s=a195ca612dd64dac5ffb27c8ea9d0b32', 'https://myanimelist.net/anime/19/Monster']
[26, 'Jujutsu Kaisen 2nd Season', 8.85, 0, 647811, 'TV', 23, '', '', datetime.date(2023, 7, 1), '', 'https://cdn.myanimelist.net/r/100x140/images/anime/1792/138022.jpg?s=b279b7d184058e92af5638e324451b95', 'https://myanimelist.net/anime/51009/Jujutsu_Kaisen_2nd_Season']


-------------Processing Page----------:   0%|          | 0/1 [00:04<?, ?item/s]


ValueError: invalid literal for int() with base 10: '?'

In [150]:
Row_Master 

[[4,
  'Gintama: The Final',
  9.05,
  1551,
  140720,
  'Movie',
  1,
  ['Action', 'Comedy', 'Drama', 'Sci-Fi'],
  'Shounen',
  datetime.date(2021, 1, 1),
  datetime.date(2021, 1, 1),
  'https://cdn.myanimelist.net/r/100x140/images/anime/1245/116760.jpg?s=8fc5f4b436ee45307a882948a5e768ac',
  'https://myanimelist.net/anime/39486/Gintama__The_Final'],
 [4,
  'Gintama: The Final',
  9.05,
  1551,
  140720,
  'Movie',
  1,
  ['Action', 'Comedy', 'Drama', 'Sci-Fi'],
  'Shounen',
  datetime.date(2021, 1, 1),
  datetime.date(2021, 1, 1),
  'https://cdn.myanimelist.net/r/100x140/images/anime/1245/116760.jpg?s=8fc5f4b436ee45307a882948a5e768ac',
  'https://myanimelist.net/anime/39486/Gintama__The_Final'],
 [4,
  'Gintama: The Final',
  9.05,
  1551,
  140720,
  'Movie',
  1,
  ['Action', 'Comedy', 'Drama', 'Sci-Fi'],
  'Shounen',
  datetime.date(2021, 1, 1),
  datetime.date(2021, 1, 1),
  'https://cdn.myanimelist.net/r/100x140/images/anime/1245/116760.jpg?s=8fc5f4b436ee45307a882948a5e768ac',
  

In [94]:
for row in Row_Master:
    df = pd.concat([df, pd.Series(row)], ignore_index=True)

In [95]:
df.head()

,Rank,Title,Score,popularity,Members,Type,Episodes,Genre,Demographic,Start Date,End Date,Image,Link,0
0,1,Fullmetal Alchemist: Brotherhood,9.10,3,3246818,TV,64,"[Action, Adventure, Drama, Fantasy]",Shounen,2009-04-01,2010-07-01 00:00:00,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/5114/Fullmetal_A...,NaN
1,2,Steins;Gate,9.07,13,2494983,TV,24,"[Drama, Sci-Fi, Suspense]",,2011-04-01,2011-09-01 00:00:00,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/9253/Steins_Gate,NaN
2,3,Gintama°,9.06,335,611603,TV,51,"[Action, Comedy, Sci-Fi]",Shounen,2015-04-01,2016-03-01 00:00:00,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/28977/Gintama°,NaN
3,4,Gintama: The Final,9.05,1551,140720,Movie,1,"[Action, Comedy, Drama, Sci-Fi]",Shounen,2021-01-01,2021-01-01 00:00:00,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/39486/Gintama__T...,NaN
4,5,Shingeki no Kyojin Season 3 Part 2,9.05,22,2173977,TV,10,"[Action, Drama, Suspense]",Shounen,2019-04-01,2019-07-01 00:00:00,https://cdn.myanimelist.net/r/100x140/images/a...,https://myanimelist.net/anime/38524/Shingeki_n...,NaN


In [96]:
df.shape

(16860, 14)

In [101]:
df.columns

Index([       'Rank',       'Title',       'Score',  'popularity',
           'Members',        'Type',    'Episodes',       'Genre',
       'Demographic',  'Start Date',    'End Date',       'Image',
              'Link',             0],
      dtype='object')

In [103]:
df = df.drop(0, axis=1)

In [126]:
df.iloc[14]

Rank                                                           1
Title                           Fullmetal Alchemist: Brotherhood
Score                                                       9.10
popularity                                                     3
Members                                                  3246818
Type                                                          TV
Episodes                                                      64
Genre                        [Action, Adventure, Drama, Fantasy]
Demographic                                              Shounen
Start Date                                   2009-04-01 00:00:00
End Date                                     2010-07-01 00:00:00
Image          https://cdn.myanimelist.net/r/100x140/images/a...
Link           https://myanimelist.net/anime/5114/Fullmetal_A...
Name: 14, dtype: object

In [153]:

table = soup[18].find('table',class_='top-ranking-table')
row_data = soup[18].find_all('tr', class_='ranking-list')

### Initializing default values in case Data is missing
rank = 0
title = ''
score = 0
popularity=0
members=0
anime_type=''
episodes=0
genre=''
demographic=''
start_date='' 
end_date='' 
img_value=''
href_value=''


rd = row_data[48].find_all('td')

rank_tag = rd[0].find('span', class_ = 'lightLink')
if rank_tag:
    rank =rank_tag.text 
    rank = int(rank)

anchor_tag = rd[1].find('a')
if anchor_tag:
    href_value = anchor_tag.get('href')



img_tag = rd[1].find('img')
if img_tag:
    img_value = img_tag.get('data-srcset').split(", ")
    img_value = [img[:-3] for img in img_value]
    img_value = img_value[1]

name_div = rd[1].find('div', class_='di-ib')
if name_div:
        title = name_div.find('a',class_='hoverinfo_trigger').text
        title = title.replace('"', '')


detail_div = rd[1].find('div', class_='information')
if detail_div:
        details = detail_div.text.split('\n')
        details = [det.strip() for det in details if det.strip()!='']



score_div = rd[2].find('span')
if score_div:
        score =score_div.text 
        score = float(score)

res = requests.get(href_value)
if res.status_code == 200:
    anime_soup = BeautifulSoup(res.text, 'html')
    table = anime_soup.find('table')
    table_row = table.find('tr')
    pop_div = table_row.find('div', class_='stats-block po-r clearfix')

    strong = pop_div.find('span', class_='numbers popularity')
    strong = strong.find_all('strong')
    popularity = strong[0].text[1:]
    popularity = int(popularity)

    left_div = anime_soup.find('div', class_='leftside')
    info_div = left_div.find_all('div', class_='spaceit_pad')

    for info in info_div:

        text = info.find('span').text
        if text == 'Demographic:':
             demographic = info.find_all('span')[1].text
        elif text == 'Genres:' or text == 'Genre:':
            genre = info.find_all('span')
            genres = []
            for gen in genre:
                genres.append(gen.text)
            genre = [gen.strip() for gen in genres]
            genre=genre[1:]

members = details[2]
members = ''.join(filter(str.isdigit, members))
members = int(members)


anime_type_eps = details[0].split(' ')
anime_type = anime_type_eps[0]
episodes = anime_type_eps[1][1:]
episodes = int(episodes)

if "to" in details[1]:
    date_string = details[1]
    start_date, end_date = date_string.split(' to ')
else:
    date_string = details[1]
    if len(date_string.split(' - ')) == 1:
        start_date,end_date = date_string.split(' -')
        start_date = datetime.strptime(start_date, '%b %Y').date()

    else:
        start_date, end_date = date_string.split(' - ')
        start_date = datetime.strptime(start_date, '%b %Y').date()
        end_date = datetime.strptime(end_date, '%b %Y').date()

new_list = [rank,title,score,popularity, members,anime_type, episodes, genre, demographic,start_date, end_date, img_value, href_value]

print(new_list)

[949, 'City Hunter 3', 7.8, 0, 21147, 'TV', 13, '', '', datetime.date(1989, 10, 1), datetime.date(1990, 1, 1), 'https://cdn.myanimelist.net/r/100x140/images/anime/1147/94393.jpg?s=8d251b0b525b7754f8b9dc8bba41901d', 'https://myanimelist.net/anime/1472/City_Hunter_3']
